# 协程
- 参考资料
    - http://python.jobbole.com/86481/
    - http://python.jobbole.com/87310/
    - https://segmentfault.com/a/1190000009781688

# 迭代器
- 可迭代(Iterable):直接作用于for循环的变量
- 迭代器(Iterator):不但可以作用于for循环，还可以被next调用
- list是典型的可迭代对象，但不是迭代器
- 通过isinstance判断
- iterable和iterator可以转换
    - 通过iter函数
- 注：迭代器在操作时会自动调用next()，因此操作后迭代器的内容会慢慢失去

In [6]:
# 可迭代
l = [ i for i in range(10)]

# l是可迭代的，但不是迭代器
for idx in l:
    print(idx,end="")

# range是个迭代器，用完以后就没了，被next调用
for i in range(5):
    print(i,end="")

012345678901234

In [12]:
# isinstance案例
# 判断某个变量是否是某个类的实例

# 判断是否课可迭代
from collections import Iterable
ll = [1,2,3,4,5]

print(isinstance(ll, Iterable))

from collections import Iterator
print(isinstance(ll, Iterator))

True
False


In [9]:
# iter函数

s = 'i love wangxiaojign'

print(isinstance(s, Iterable))
print(isinstance(s, Iterator))

s_iter = iter(s)
print(isinstance(s_iter, Iterable))
print(isinstance(s_iter, Iterator))

True
False
True
True


# 生成器
- generator: 一边循环一边计算下一个元素的机制/算法(避免超长的列表，内容无法满足)，用一个扔一个，以时间换空间
- 需要满足三个条件：
    - 每次调用都生产出for循环需要的下一个元素或者
    - 如果达到最后一个后，爆出StopIteration异常
    - 可以被next函数调用
- 如何生成一个生成器
    - 直接使用
    - 如果函数中包含yield，则这个函数就叫生成器
    - next调用函数，遇到yield返回

In [13]:
# 直接使用生成器

L = [x*x for x in range(5)] # 放在中括号中是列表生成器
g = (x*x for x in range(5))#  放在小括号中就是生成器

print(type(L))
print(type(g))

<class 'list'>
<class 'generator'>


In [14]:
# 函数案例

def odd():
    print("Step 1")
    print("Step 2")
    print("Step 3")
    return None

odd()

Step 1
Step 2
Step 3


In [15]:
# 生成器的案例
# 在函数odd中，yield负责返回
def odd():
    print("Step 1")
    yield 1 # 返回1，暂时停在这里，记住了这个地址，下次next()就从这里开始
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3
 
# odd() 是调用生成器
g = odd() # 生成一个生成器
one = next(g) # next()执行生成器的下一步
print(one)

two = next(g)
print(two)

three = next(g)
print(three)

Step 1
1
Step 2
2
Step 3
3


In [16]:
# for循环调用生成器
def fib(max):
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        print(b)
        a,b = b, a+b # 注意写法
        n += 1
        
    return 'Done'

fib(5)

1
1
2
3
5


'Done'

In [17]:
# 斐波那契额数列的生成器写法
def fib(max):
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        yield b 
        a,b = b, a+b # 注意写法
        n += 1
        
    #需要注意，爆出异常是的返回值是return的返回值
    return 'Done'

g = fib(5)

for i in range(6):
    rst = next(g) # 最后一次调用，已经进不到yield了，只能从return出来，因此出现异常
    print(rst)

1
1
2
3
5


StopIteration: Done

In [18]:
ge = fib(10)
'''
生成器的典型用法是在for中使用
比较常用的典型生成器就是range
'''
for i in ge: #在for循环中使用生成器
    print(i) # 循环中收到的值就是yield的返回值

1
1
2
3
5
8
13
21
34
55


# 协程
- 相比于大量切换极其消耗资源的多线程，协程的切换消耗的资源非常少，适合同时服务于大量同时的请求
- 利用yield的返回，将线程切换变成了每次执行一个函数的一小部分
- 历史历程
    - 3.4引入协程，用yield实现
    - 3.5引入协程语法
    - 实现的协程比较好的包有asyncio, tornado, gevent
- 定义：协程 是为非抢占式多任务产生子程序的计算机程序组件，协程允许不同入口点在不同位置暂停或开始执行程序”。
- 从技术角度讲，协程就是一个你可以暂停执行的函数，或者干脆把协程理解成生成器
- 协程的实现：
    - yield返回
    - send调用
- 协程的四个状态
    - inspect.getgeneratorstate(…) 函数确定，该函数会返回下述字符串中的一个：
    - GEN_CREATED：等待开始执行
    - GEN_RUNNING：解释器正在执行
    - GEN_SUSPENED：在yield表达式处暂停
    - GEN_CLOSED：执行结束
    - next预激（prime)
    - 代码案例v2
- 协程终止
    - 协程中未处理的异常会向上冒泡，传给 next 函数或 send 方法的调用方（即触发协程的对象）
    - 终止协程的一种方式：发送某个哨符值，让协程退出。内置的 None 和Ellipsis 等常量经常用作哨符值==。
- yield from
    - 调用协程为了得到返回值，协程必须正常终止
    - 生成器正常终止会发出StopIteration异常，异常对象的vlaue属性保存返回值
    - yield from从内部捕获StopIteration异常
    - 案例v03
    - 委派生成器
        - 包含yield from表达式的生成器函数
        - 委派生成器在yield from表达式出暂停，调用方可以直接把数据发给子生成器
        - 子生成器在把产出的值发给调用方
        - 子生成器在最后，解释器会抛出StopIteration，并且把返回值附加到异常对象上
        - 案例v04

In [2]:
# 协程实现案例1
# 协程代码案例1
def simple_coroutine():
    print('-> start')
    x = yield # 返回None，后面时空的    这里是先直接返回为空，下次进来时一个赋值
    print('-> recived', x)

#主线程
sc = simple_coroutine() # 生成一个协程
print(1111)
# 可以使用sc.send(None)，效果一样
next(sc) #预激

print(2222)
sc.send('zhexiao') # 这个是发送一个参数给函数里的x（yield前面那个）

1111
-> start
2222
-> recived zhexiao


StopIteration: 

In [3]:
# 案例v2，  协程的状态
def simple_coroutine(a):
    print('-> start')

    b = yield a
    print('-> recived', a, b)

    c = yield a + b
    print('-> recived', a, b, c)

# runc
sc = simple_coroutine(5)

aa = next(sc)
print(aa)
bb = sc.send(6) # 5, 6
print(bb)
cc = sc.send(7) # 5, 6, 7
print(cc)

-> start
5
-> recived 5 6
11
-> recived 5 6 7


StopIteration: 

In [4]:
# 案例v03
def gen():
    for c in 'AB':
        yield c
# list直接用生成器作为参数
print(list(gen()))

def gen_new():
    yield from 'AB' # yield from 就是主程序和协程之间传递消息的通道
    # yield左边连接的是主程，右边就是要生成器化的'AB'
print(list(gen_new()))
# 因此最后结果一样

['A', 'B']
['A', 'B']


In [6]:
# 案例v04， 委派生成器
from collections import namedtuple

'''
解释：
1. 外层 for 循环每次迭代会新建一个 grouper 实例，赋值给 coroutine 变量； grouper 是委派生成器。
2. 调用 next(coroutine)，预激委派生成器 grouper，此时进入 while True 循环，调用子生成器 averager 后，在 yield from 表达式处暂停。
3. 内层 for 循环调用 coroutine.send(value)，直接把值传给子生成器 averager。同时，当前的 grouper 实例（coroutine）在 yield from 表达式处暂停。
4. 内层循环结束后， grouper 实例依旧在 yield from 表达式处暂停，因此， grouper函数定义体中为 results[key] 赋值的语句还没有执行。
5. coroutine.send(None) 终止 averager 子生成器，子生成器抛出 StopIteration 异常并将返回的数据包含在异常对象的value中，yield from 可以直接抓取 StopItration 异常并将异常对象的 value 赋值给 results[key]
'''
ResClass = namedtuple('Res', 'count average')


# 子生成器
def averager(): 
    total = 0.0
    count = 0
    average = None

    while True:
        term = yield
        # None是哨兵值
        if term is None:
            break
        total += term
        count += 1
        average = total / count

    return ResClass(count, average)


# 委派生成器
def grouper(storages, key):
    while True:
        # 获取averager()返回的值
        storages[key] = yield from averager()


# 客户端代码
def client():
    process_data = {
        'boys_2': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys_1': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
    }

    storages = {}
    for k, v in process_data.items():
        # 获得协程
        coroutine = grouper(storages, k)

        # 预激协程
        next(coroutine)

        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)

        # 终止协程
        coroutine.send(None)
    print(storages)

# run
client()

{'boys_2': Res(count=9, average=40.422222222222224), 'boys_1': Res(count=9, average=1.3888888888888888)}


# asyncio
- python3.4开始引入的标准库,内置了对异步io的支持
- asyncio本身是一个消息循环,
- 步骤 
    - 创建消息循环
    - 把协程导入
    - 关闭
案例08
案例09-两个tasks
案例v10-得到多个网站

In [1]:
import threading
# 引入异步IO包
import asyncio

# 使用协程
@asyncio.coroutine
def hello():
    print("Hello World!(%s)" % threading.currentThread())
    print("Start ...(%s)" % threading.currentThread())
    yield from asyncio.sleep(10)
    print("Done ...(%s)" % threading.currentThread())
    print("Hello again!(%s)" % threading.currentThread())

# 启动消息循环loop
loop = asyncio.get_event_loop()
# 定义两个任务，这里是协程
tasks = [hello(), hello()]
# asyncio使用wait等待task执行完毕
loop.run_until_complete(asyncio.wait(tasks))
# 关闭消息队列
loop.close()

RuntimeError: This event loop is already running

Hello World!(<_MainThread(MainThread, started 2124)>)
Start ...(<_MainThread(MainThread, started 2124)>)
Hello World!(<_MainThread(MainThread, started 2124)>)
Start ...(<_MainThread(MainThread, started 2124)>)
Done ...(<_MainThread(MainThread, started 2124)>)
Hello again!(<_MainThread(MainThread, started 2124)>)
Done ...(<_MainThread(MainThread, started 2124)>)
Hello again!(<_MainThread(MainThread, started 2124)>)


# async and await
- 为了更好的表示异步io
- python3.5 开始引入
- 让coroutine代码更简洁
- 使用上,可以简单进行替换 
    - 可以把 @asyncio.coroutine 替换成async
    - yield from 替换成await

# aiohttp
- 介绍 
- asyncio实现单线程并发IO,在客户端用处不大
- 在服务器端可以asyncio+coroutine配合,因为http是io操作
- asyncio实现了TCP,UIDP,SSL等协议
- aiohttp是给予asyncio实现的HTTP框架
- pip install aiohttp

# concurrent.futures
- python3新增的库
- 类似其他语言的线程池的概念
- 此模块利用multiprocessiong实现真正的并行计算
- 核心原理是：concurrent.futures会以子进程的形式，并行的运行多个python解释器，从而令python程序可以利用多核CPU来提升执行速度。 由于子进程与主解释器相分离，所以他们的全局解释器锁也是相互独立的。每个子进程都能够完整的使用一个CPU内核。
- concurrent.futures.Executor
    - ThreadPoolExecutor
    - ProcessPoolExecutor
    - 执行时要自行选择
- submit(fn, args, kwargs) 要提交进行并行的工作fn  done result
    - fn:异步执行的函数
    - args,kwargs:参数

In [ ]:
- map(fn, *iterables, timeout=None)
    - 跟map函数类似
    - 函数需要异步执行
    - timeout: 超时时间
    - map和submit使用一个就行 
- Future
    - 未来需要完成的任务
    - future 实例由Excutor.submit创建